In [ ]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.llms import CTransformers


#Extract Text from .txt Files

def extract_text_from_txt(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return f.read()

#Load Documents

def load_documents(folder_path="documents"):
    documents = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".txt"):
            file_path = os.path.join(folder_path, file_name)
            text = extract_text_from_txt(file_path)
            documents.append({"file_name": file_name, "text": text})
    return documents

#Split Documents

def split_documents(documents, chunk_size=500, chunk_overlap=50):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = []
    for doc in documents:
        chunks.extend(splitter.split_text(doc["text"]))
    return [Document(page_content=chunk) for chunk in chunks]


#Embedding and FAISS Creation

def create_faiss_index(documents):
    embedding_model = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={"device": "cpu"}
    )
    vectordb = FAISS.from_documents(documents, embedding_model)
    vectordb.save_local("faiss_index")
    print("✅ FAISS index created and saved.")
    return vectordb

#Load FAISS Index

def load_faiss_index():
    embedding_model = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={"device": "cpu"}
    )
    vectordb = FAISS.load_local("faiss_index", embeddings=embedding_model, allow_dangerous_deserialization=True)
    print("✅ FAISS index loaded.")
    return vectordb

#Retrieve Relevant Chunks

def retrieve_relevant_chunks(vectordb, query, top_k=3):
    results = vectordb.similarity_search(query, k=top_k)
    context = "\n\n".join([result.page_content for result in results])
    return context

#Answer Generation (Llama 2)

def generate_answer(context, question):
    llm_model = CTransformers(
        model=r"" \
        "",
        model_type="llama",
        config={
            "max_new_tokens": 256,
            "temperature": 0.7,
            "context_length": 512
        }
    )
    prompt = f"Context:\n{context}\n\nQuestion: {question}\n\nAnswer:"
    response = llm_model(prompt)
    return response


import streamlit as st


In [2]:
# Function to extract text from .txt
def extract_text_from_txt(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return f.read()

In [3]:

# Load documents
def load_documents(folder_path="documents"):
    documents = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".txt"):
            file_path = os.path.join(folder_path, file_name)
            text = extract_text_from_txt(file_path)
            documents.append({"file_name": file_name, "text": text})
    return documents

In [4]:

# Split documents
def split_documents(documents, chunk_size=500, chunk_overlap=50):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = []
    for doc in documents:
        chunks.extend(splitter.split_text(doc["text"]))
    return [Document(page_content=chunk) for chunk in chunks]

In [5]:
# Create FAISS index
def create_faiss_index(documents):
    embedding_model = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={"device": "cpu"}
    )
    vectordb = FAISS.from_documents(documents, embedding_model)
    vectordb.save_local("faiss_index")
    return vectordb

In [6]:
# Load FAISS index
def load_faiss_index():
    embedding_model = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={"device": "cpu"}
    )
    vectordb = FAISS.load_local("faiss_index", embeddings=embedding_model, allow_dangerous_deserialization=True)
    return vectordb


In [7]:

# Retrieve relevant chunks
def retrieve_relevant_chunks(vectordb, query, top_k=3):
    results = vectordb.similarity_search(query, k=top_k)
    context = "\n\n".join([result.page_content for result in results])
    return context

In [8]:
# Generate answer
def generate_answer(context, question):
    llm_model = CTransformers(
        model=r"D:\GenAi\llama-2-7b-chat.ggmlv3.q4_0.bin",  # Your local model path
        model_type="llama",
        config={
            "max_new_tokens": 256,
            "temperature": 0.7,
            "context_length": 512
        }
    )
    prompt = f"Context:\n{context}\n\nQuestion: {question}\n\nAnswer:"
    response = llm_model(prompt)
    return response


In [13]:
import nltk

# Manually set nltk data path
nltk.data.path.append(r"C:\Users\DELL\AppData\Roaming\nltk_data")

# Download if needed
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
true_answer = "The correct response from the model"
words = true_answer.split()
print(words)


['The', 'correct', 'response', 'from', 'the', 'model']


In [15]:
from nltk.translate.bleu_score import sentence_bleu

# Your model output and correct answer
generated_answer = "Vitamin C helps boost immunity."
true_answer = "Vitamin C strengthens the immune system."

# BLEU Score Calculation
reference = [true_answer.split()]  # ✅ split instead of word_tokenize
candidate = generated_answer.split()  # ✅ split instead of word_tokenize

bleu_score = sentence_bleu(reference, candidate)
print(f"BLEU Score: {bleu_score:.4f}")


BLEU Score: 0.0000


c:\Users\DELL\anaconda3\envs\Saralads\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\DELL\anaconda3\envs\Saralads\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [16]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Smoothing function
smooth_fn = SmoothingFunction().method1

bleu_score = sentence_bleu(reference, candidate, smoothing_function=smooth_fn)
print(f"BLEU Score with smoothing: {bleu_score:.4f}")


BLEU Score with smoothing: 0.0930
